# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from my_secrets import IEX_CLOUD_API_TOKEN
print(IEX_CLOUD_API_TOKEN)

pk_9e0e17017ccd4a0b931feed227bfbd81


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [9]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/aapl?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()

#type(data)
#dir(data)
#print(data.status_code)
print(data)

[{'avgTotalVolume': 53551384, 'calculationPrice': 'close', 'change': -1.05, 'changePercent': -0.00542, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': 'Close', 'highTime': 1703883600433, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 192.62, 'iexCloseTime': 1703883599011, 'iexLastUpdated': 1703883599011, 'iexMarketPercent': 3.3979393458352383, 'iexOpen': 194, 'iexOpenTime': 1703860200778, 'iexRealtimePrice': 192.62, 'iexRealtimeSize': 9, 'iexVolume': 676404, 'lastTradeTime': 1703883599600, 'latestPrice': 192.53, 'latestSource': 'Close', 'latestTime': 'December 29, 2023', 'latestUpdate': 1703883600433, 'latestVolume': None, 'low': None, 'lowSource': 'Close', 'lowTime': 1703883600000, 'marketCap': 2994371342560, 'odd

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
type(data)
price = data[0]['latestPrice']
print(price)
market_cap = data[0]['marketCap']
print(market_cap/1000000000000)

192.53
2.99437134256


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [39]:
stock_lst = []
stock_lst.append([symbol, price, market_cap, 'N/A'])



,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,AAPL,192.53,2994371342560,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [42]:
stock_lst = []


final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    price = data[0]['latestPrice']
    market_cap = data[0]['marketCap']
    stock_lst.append([stock, price, market_cap, 'N/A'])


[['A', 139.03, 40736360301, 'N/A'], ['AAL', 13.74, 8979647157, 'N/A'], ['AAP', 61.03, 3631406450, 'N/A'], ['AAPL', 192.53, 2994371342560, 'N/A'], ['ABBV', 154.97, 278946000000, 'N/A'], ['ABC', 179.98, 36387430583, 'N/A'], ['ABMD', 381.02, 17180642928, 'N/A'], ['ABT', 110.07, 191087963058, 'N/A'], ['ACN', 350.91, 222826362493, 'N/A'], ['ADBE', 596.6, 271631980000, 'N/A'], ['ADI', 198.56, 98537718784, 'N/A'], ['ADM', 72.22, 38520794236, 'N/A'], ['ADP', 232.97, 95821675063, 'N/A'], ['ADSK', 243.48, 52084103331, 'N/A'], ['AEE', 72.34, 19021444772, 'N/A'], ['AEP', 81.22, 42711618912, 'N/A'], ['AES', 19.25, 12890358924, 'N/A'], ['AFL', 82.5, 48211340843, 'N/A'], ['AIG', 67.75, 47563233238, 'N/A'], ['AIV', 7.83, 1141298232, 'N/A'], ['AIZ', 168.49, 8860993564, 'N/A'], ['AJG', 224.88, 48551592000, 'N/A'], ['AKAM', 118.35, 17850916073, 'N/A'], ['ALB', 144.48, 16955200739, 'N/A'], ['ALGN', 274, 20985417236, 'N/A'], ['ALK', 39.07, 5003033718, 'N/A'], ['ALL', 139.98, 36630955219, 'N/A'], ['ALLE', 1

In [43]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy' ] 
final_dataframe = pd.DataFrame(stock_lst, columns = my_columns)

final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,139.03,4.073636e+10,N/A
1,AAL,13.74,8.979647e+09,N/A
2,AAP,61.03,3.631406e+09,N/A
3,AAPL,192.53,2.994371e+12,N/A
4,ABBV,154.97,2.789460e+11,N/A
...,...,...,...,...
500,YUM,130.66,3.662507e+10,N/A
501,ZBH,121.70,2.543295e+10,N/A
502,ZBRA,273.33,1.403812e+10,N/A
503,ZION,43.87,6.499288e+09,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [57]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy' ] 
stock_lst = []

# stocks['Ticker'] is read in from the PDF
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):    
    symbol_strings.append(','.join(symbol_groups[i]))

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'    
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        for item in data:
            if item['symbol'] == symbol:
                stock_lst.append([symbol, item['latestPrice'], item['marketCap'], 'N/A'])       

final_dataframe = pd.DataFrame(stock_lst, columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,139.990,4.101764e+10,N/A
1,AAL,13.430,8.777050e+09,N/A
2,AAP,61.390,3.652827e+09,N/A
3,AAPL,186.420,2.899344e+12,N/A
4,ABBV,157.860,2.841480e+11,N/A
...,...,...,...,...
500,YUM,130.360,3.654098e+10,N/A
501,ZBH,122.175,2.553222e+10,N/A
502,ZBRA,266.650,1.369504e+10,N/A
503,ZION,43.860,6.497806e+09,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.


We'll do that next.

In [71]:
portfolio_size = float(input('Enter the value of your portfolio: '))


Enter the value of your portfolio:  10000000


In [77]:
position_size = portfolio_size / int(len(final_dataframe.index))
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe
# num_aapl_shares = position_size / 500 
# print(math.floor(num_aapl_shares))

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,139.990,4.101764e+10,141
1,AAL,13.430,8.777050e+09,1474
2,AAP,61.390,3.652827e+09,322
3,AAPL,186.420,2.899344e+12,106
4,ABBV,157.860,2.841480e+11,125
...,...,...,...,...
500,YUM,130.360,3.654098e+10,151
501,ZBH,122.175,2.553222e+10,162
502,ZBRA,266.650,1.369504e+10,74
503,ZION,43.860,6.497806e+09,451


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [106]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [107]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,        
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": '$0.00',
        "font_color": font_color,
        "bg_color": background_color,        
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": '0',
        "font_color": font_color,
        "bg_color": background_color,        
        "border": 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [104]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.close()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalisation', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [108]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalisation', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()